<a href="https://colab.research.google.com/github/allanVvz/Notebook-Sandbox/blob/main/ElizaBr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Imports e utilidades
import re
import random
from typing import List, Tuple, Pattern


In [2]:
# Reflexões/pronomes e helpers
REFLECTIONS = {
    "i": "you", "me": "you", "my": "your", "mine": "yours", "am": "are",
    "myself": "yourself", "you": "I", "your": "my", "yours": "mine", "yourself": "myself",
    "i'm": "you're", "i’d": "you’d", "i've": "you've", "i'll": "you'll",
    "you're": "I'm", "you’d": "I’d", "you've": "I've", "you'll": "I'll",
    "eu": "você", "mim": "você", "meu": "seu", "minha": "sua", "meus": "seus", "minhas": "suas",
    "sou": "é", "estou": "está", "vou": "vai",
    "você": "eu", "seu": "meu", "sua": "minha", "seus": "meus", "suas": "minhas",
}

def reflect(fragment: str) -> str:
    tokens = re.findall(r"\w+|[^\w\s]", fragment, flags=re.UNICODE)
    out = []
    for t in tokens:
        lower = t.lower()
        if lower in REFLECTIONS:
            rep = REFLECTIONS[lower]
            if t[0].isupper():
                rep = rep.capitalize()
            out.append(rep)
        else:
            out.append(t)
    s = ""
    for i, tok in enumerate(out):
        if i > 0 and re.match(r"\w", tok) and re.match(r"\w", out[i-1]):
            s += " " + tok
        elif i > 0 and re.match(r"\w", tok) and re.match(r"[\)\]\}\!\?.,;:]", out[i-1]):
            s += " " + tok
        else:
            s += tok if i == 0 else (" " + tok if re.match(r"\w", tok) else tok)
    return s.strip()

def clean(text: str) -> str:
    return re.sub(r"\s+", " ", text.strip())


In [3]:
# Regras com padrões de regex
RAW_RULES: List[Tuple[str, List[str]]] = [
    (r"\bI need\s+(.+)", [
        "Why do you need {0}?",
        "Would it really help you to get {0}?",
        "What would it mean to you if you had {0}?"
    ]),
    (r"\bI want\s+(.+)", [
        "Why do you want {0}?",
        "What would change if you got {0}?"
    ]),
    (r"\bI(?:'m| am)\s+(.+)", [
        "How long have you been {0}?",
        "Why do you think you are {0}?"
    ]),
    (r"\bI feel\s+(.+)", [
        "Tell me more about feeling {0}.",
        "What makes you feel {0}?"
    ]),
    (r"\b(?:mother|father|family|friend|partner|wife|husband)\b.*", [
        "Tell me more about your relationships.",
        "How does your family influence this situation?"
    ]),
    (r"(.+)\?\s*$", [
        "What do you think?",
        "Why do you ask that?"
    ]),
    # Versões em português
    (r"\beu preciso(?: de)?\s+(.+)", [
        "Por que você precisa de {0}?",
        "Ter {0} realmente ajudaria?",
        "O que mudaria se você tivesse {0}?"
    ]),
    (r"\beu quero\s+(.+)", [
        "Por que você quer {0}?",
        "O que mudaria se você conseguisse {0}?"
    ]),
    (r"\b(?:eu estou|estou|sou)\s+(.+)", [
        "Há quanto tempo você está {0}?",
        "Por que você acha que está {0}?"
    ]),
    (r"\beu me sinto\s+(.+)", [
        "Conte mais sobre sentir {0}.",
        "O que te faz sentir {0}?"
    ]),
    (r"(.+)\?\s*$", [
        "O que você acha?",
        "Por que essa pergunta?"
    ]),
]

RULES: List[Tuple[Pattern, List[str]]] = [(re.compile(p, re.IGNORECASE), resps) for p, resps in RAW_RULES]

DEFAULT_RESPONSES = [
    "Please, go on.",
    "Can you elaborate?",
    "Interesting—tell me more.",
    "Entendo. Pode detalhar um pouco mais?"
]


In [4]:
# Motor de resposta
def eliza_reply(text: str) -> str:
    text = clean(text)
    for pattern, responses in RULES:
        m = pattern.search(text)
        if m:
            groups = [reflect(clean(g)) for g in m.groups()]
            template = random.choice(responses)
            try:
                return template.format(*groups)
            except Exception:
                return random.choice(responses)
    return random.choice(DEFAULT_RESPONSES)


In [10]:
# Testes rápidos
tests = [
    "I need food",
    "I want a break",
    "I'm tired of work",
    "I feel anxious",
    "My mother keeps calling me",
    "Why is this happening?",
    "eu preciso de ajuda",
    "eu quero descansar",
    "eu estou com medo",
    "eu me sinto cansado",
    "isso faz sentido?"
]

for t in tests:
    print(f"\nUser: {t}\nELIZA: {eliza_reply(t)}")



User: I need food
ELIZA: Would it really help you to get food?

User: I want a break
ELIZA: What would change if you got a break?

User: I'm tired of work
ELIZA: How long have you been tired of work?

User: I feel anxious
ELIZA: Tell me more about feeling anxious.

User: My mother keeps calling me
ELIZA: How does your family influence this situation?

User: Why is this happening?
ELIZA: What do you think?

User: eu preciso de ajuda
ELIZA: O que mudaria se você tivesse ajuda?

User: eu quero descansar
ELIZA: Por que você quer descansar?

User: eu estou com medo
ELIZA: Há quanto tempo você está com medo?

User: eu me sinto cansado
ELIZA: O que te faz sentir cansado?

User: isso faz sentido?
ELIZA: What do you think?


In [7]:
# Loop de chat interativo
def chat():
    print("ELIZA: Hello, I'm here to talk. How are you today?")
    while True:
        try:
            user = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nELIZA: Goodbye!")
            break
        if user.lower() in {"quit", "exit", "sair"}:
            print("ELIZA: Goodbye!")
            break
        print("ELIZA:", eliza_reply(user))

# Para iniciar a conversa:
# chat()


In [ ]:
chat()

ELIZA: Hello, I'm here to talk. How are you today?
You: My wife is distant lately
ELIZA: How does your family influence this situation?
You: It makes me feel worthless
ELIZA: Can you elaborate?
You: I don’t think I deserve love
ELIZA: Can you elaborate?
